In [1]:
from pathlib import Path
import json
import sys

import gtfstk as gt
import pandas as pd
import numpy as np
import shapely.geometry as sg
import folium

sys.path.append('../')

import gtfs_map_matcher as mm

%load_ext autoreload
%autoreload 2

DATA_DIR = Path('../data')
OUT_DIR = Path('../output')
SECRETS_PATH = Path('../secrets.json')

In [13]:
# # Restrict Auckland GTFS feed to a few routes for testing

# path = DATA_DIR/'auckland_gtfs_20171020.zip'
# akl = gt.read_gtfs(path, dist_units='km')

# # Select routes

# def my_select(group):
#     return group.iloc[:5]

# new_routes = akl.routes.groupby('route_type', group_keys=False).apply(my_select)

# akl_r = akl.restrict_to_routes(new_routes.route_id)

# path = DATA_DIR/'auckland_gtfs_sample.zip'
# gt.write_gtfs(akl_r, path)

# akl_r.assess_quality()

In [15]:
points_by_key = {
    'bingo': [
        [174.843234, -41.137425],
        [174.828152, -41.130639],
    ],
    'bongo': [
        [174.80496399999998, -41.22333],
        [174.796785, -41.247057],
    ]
}

mm.map_match_google(points_by_key, mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'))

{'warningMessage': "Input path is too sparse. You should provide a path where consecutive points are closer to each other. Refer to the 'path' parameter in Google Roads API documentation.", 'snappedPoints': [{'placeId': 'ChIJ8dqN_LetOG0RzRAx6EPYfqQ', 'originalIndex': 0, 'location': {'latitude': -41.22335545870424, 'longitude': 174.80501397147776}}, {'placeId': 'ChIJTcHE8XSuOG0RQLWtBmHvABM', 'originalIndex': 1, 'location': {'latitude': -41.246993559684476, 'longitude': 174.7967170172291}}]}
{'warningMessage': "Input path is too sparse. You should provide a path where consecutive points are closer to each other. Refer to the 'path' parameter in Google Roads API documentation.", 'snappedPoints': [{'placeId': 'ChIJUzPxV25TP20RgBgWB2HvABM', 'originalIndex': 0, 'location': {'latitude': -41.137424437287834, 'longitude': 174.8431671586028}}, {'placeId': 'ChIJoYfK2A5TP20RobqrBmHvABM', 'originalIndex': 1, 'location': {'latitude': -41.13063985048523, 'longitude': 174.82814702694546}}]}


{'bingo': [[174.8431671586028, -41.137424437287834],
  [174.82814702694546, -41.13063985048523]],
 'bongo': [[174.80501397147776, -41.22335545870424],
  [174.7967170172291, -41.246993559684476]]}

In [2]:
path = DATA_DIR/'auckland_gtfs_sample.zip'
feed = gt.read_gtfs(path, dist_units='km')
feed.describe()

,indicator,value
0,agencies,"[Go West, AT Metro, Fullers Ferries, Pavlovich..."
1,timezone,Pacific/Auckland
2,start_date,20171012
3,end_date,20171220
4,num_routes,15
5,num_trips,4675
6,num_stops,1022
7,num_shapes,195
8,sample_date,20171019
9,num_routes_active_on_sample_date,15


In [3]:
# Create sample points

t = feed.trips.merge(feed.routes)
cond = t['route_type'] == 3
t = t[cond].copy()
%time spoints_1 = mm.sample_trip_points(feed, t.trip_id, 5)
%time spoints_2 = mm.sample_trip_points(feed, t.trip_id, 100)
%time spoints_3 = mm.sample_trip_points(feed, t.trip_id, point_dist=0.1)

print('num stop patterns = ', len(spoints_2))
pattern = list(spoints_1.keys())[0]


CPU times: user 548 ms, sys: 0 ns, total: 548 ms
Wall time: 552 ms
CPU times: user 752 ms, sys: 0 ns, total: 752 ms
Wall time: 747 ms
CPU times: user 980 ms, sys: 0 ns, total: 980 ms
Wall time: 971 ms
num stop patterns =  50


In [9]:
l1 = sg.LineString(spoints_1[pattern])
l2 = sg.LineString(spoints_2[pattern])
l3 = sg.LineString(spoints_3[pattern])

# Get actual trip shape
p = mm.get_stop_patterns(feed)
shid = p.loc[p['stop_pattern'] == pattern, 'shape_id'].unique()[0]
la = feed.build_geometry_by_shape(shape_ids=[shid], use_utm=False)[shid]

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(l1), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
folium.GeoJson(sg.mapping(l2), 
  style_function=lambda x: {'color': 'orange'}).add_to(mappy)
folium.GeoJson(sg.mapping(l3), 
  style_function=lambda x: {'color': 'yellow'}).add_to(mappy)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
mappy

In [10]:
# Map match with Mapzen

mpoints = mm.map_match_mapzen({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'MAPZEN_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [22]:
# Map match with OSRM

mpoints = mm.map_match_osrm({pattern: spoints_2[pattern]})[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [23]:
# Map match with Mapbox, which uses OSRM

mpoints = mm.map_match_mapbox({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'MAPBOX_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [25]:
# Map match with Google

mpoints = mm.map_match_google({pattern: spoints_2[pattern]},
  mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'))[pattern]
lm = sg.LineString(mpoints)

mappy = folium.Map(location=spoints_1[pattern][0][::-1], zoom_start=14)
folium.GeoJson(sg.mapping(la), 
  style_function=lambda x: {'color': 'green'}).add_to(mappy)
folium.GeoJson(sg.mapping(lm), 
  style_function=lambda x: {'color': 'red'}).add_to(mappy)
mappy


In [5]:
# Test batch map match

n = 30
print('num requests =', n)
test = {k: v for k, v in list(spoints_2.items())[:n]}
#api_key = mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY')
api_key = mm.get_secret(SECRETS_PATH, 'MAPZEN_API_KEY')

%time mpoints = mm.map_match(test, 'mapzen', api_key)

num requests = 30
CPU times: user 100 ms, sys: 8 ms, total: 108 ms
Wall time: 9.24 s


In [6]:
print(mm.get_num_map_matching_calls(feed))

50


In [7]:
t = feed.trips.merge(feed.routes)
trip_id, shape_id = t.loc[t['route_type'] == 3, ['trip_id', 'shape_id']].iloc[0].values
print(trip_id, shape_id)
trip_ids = [trip_id]
%time feed2 = mm.create_shapes(feed, 'google', mm.get_secret(SECRETS_PATH, 'GOOGLE_API_KEY'), trip_ids=trip_ids)

14010099857-20171013114012_v59.18 2-20171013114012_v59.18
CPU times: user 660 ms, sys: 16 ms, total: 676 ms
Wall time: 2.43 s


In [8]:
print(feed.shapes[feed.shapes.shape_id == shape_id])
print(feed2.shapes[feed2.shapes.shape_id == shape_id])

                      shape_id  shape_pt_sequence  shape_pt_lon  shape_pt_lat
54520  2-20171013114012_v59.18                  0     174.75564     -36.84050
54521  2-20171013114012_v59.18                  1     174.75563     -36.84051
54522  2-20171013114012_v59.18                  2     174.75601     -36.84061
54523  2-20171013114012_v59.18                  3     174.75622     -36.84066
54524  2-20171013114012_v59.18                  4     174.75816     -36.84117
54525  2-20171013114012_v59.18                  5     174.75825     -36.84123
54526  2-20171013114012_v59.18                  6     174.75826     -36.84135
54527  2-20171013114012_v59.18                  7     174.75787     -36.84232
54528  2-20171013114012_v59.18                  8     174.75593     -36.84181
54529  2-20171013114012_v59.18                  9     174.75557     -36.84171
54530  2-20171013114012_v59.18                 10     174.75449     -36.84142
54531  2-20171013114012_v59.18                 11     174.75392 